In [1]:
import arviz as az
import numpy as np
import pymc as pm
from pymc.math import exp
import pandas as pd
%load_ext watermark

# Ants

Adapted from [Codes for Unit 7: ants.odc](https://www2.isye.gatech.edu/isye6420/supporting.html).

Associated lecture video: [Unit 7 Lesson 15](https://www.youtube.com/watch?v=999v5stS8jw&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=77).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/ants.csv).


The data discussed in Gotelli and Ellison (2002) provide the ant species richness (number of ant species) found in 64-square-meter sampling grids in 22 forests (coded as 1) and 22 bogs (coded as 2) surrounding the forests in  Connecticut, Massachusetts, and Vermont. The sites span 3Ê• of latitude in New England. There are 44 observations on four variables (columns in data set): 

- Ants: number of species, 
- Habitat: forests (1) and bogs (2), 
- Latitude
- Elevation: in meters above sea level.

(a) Using Poisson regression, model the number of ant species (Ants) with covariates Habitat and Elevation.  
(b) For a sampling grid unit located in a forest at the elevation of 100 m how many species the model from (a) predicts? For the model coefficients and the prediction report 95% credible  sets.

-----------------------

Just going to throw this up real quick because it's relevant to homework 5! Will add more in the future. Note that I've switched to PyMC 4 beta 3 here.

In [2]:
%watermark --iversions

pandas: 1.4.1
numpy : 1.22.3
pymc  : 4.0.0b4
arviz : 0.11.4



In [3]:
data = pd.read_csv("./data/ants.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   ants       44 non-null     int64
 1   habitat    44 non-null     int64
 2   elevation  44 non-null     int64
dtypes: int64(3)
memory usage: 1.2 KB


In [4]:
with pm.Model() as m:
    ant_species = pm.Data("ant_species", data["ants"].to_numpy(), mutable=False)
    habitat = pm.Data("habitat", data["habitat"].to_numpy(), mutable=True)
    elevation = pm.Data("elevation", data["elevation"].to_numpy(), mutable=True)
    
    beta0 = pm.Normal("beta0_intercept", mu=0, tau=.0001)
    beta1 = pm.Normal("beta1_habitat", mu=0, tau=.0001)
    beta2 = pm.Normal("beta2_elevation", mu=0, tau=.0001)

    μ = exp(beta0 + beta1 * habitat + beta2 * elevation)
            
    y = pm.Poisson("y", mu=μ, observed=ant_species)
    
    trace = pm.sample(
        5000,
        chains=4,
        tune=2000,
        cores=4,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0_intercept, beta1_habitat, beta2_elevation]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 2_000 tune and 5_000 draw iterations (8_000 + 20_000 draws total) took 16 seconds.
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Coul

In [5]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0_intercept,3.174,0.188,2.822,3.532,0.002,0.002,6934.0,7313.0,1.0
beta1_habitat,-0.640,0.120,-0.862,-0.412,0.001,0.001,7425.0,7886.0,1.0
beta2_elevation,-0.001,0.000,-0.002,-0.001,0.000,0.000,8855.0,8231.0,1.0


In [6]:
with m:
    pm.set_data({"habitat": [1], "elevation": [100]})
    ppc = pm.sample_posterior_predictive(trace)

az.summary(ppc)

/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:58: UserWarning: Could not extract data from symbolic observation y
  warnings.warn(f"Could not extract data from symbolic observation {obs}")
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/arviz/stats/stats.py:1180: UserWarning: Selecting first found group: posterior_predictive
  warnings.warn(f"Selecting first found group: {data.groups()[0]}")


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
y[0],10.867,3.448,5.0,17.0,0.025,0.018,18432.0,19182.0,1.0
